In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(3)

## Load Model

In [4]:
name = 'baseline_any_xse_resnet_256_seutao'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext101_32x4d(num_classes=6, pretrained=None)

In [7]:
m.avg_pool = nn.AdaptiveAvgPool2d(1)

In [8]:
sp = '~/kaggle/RSNA2019_1st_place_solution/2DCNN/src/se_resnext101_32x4d_256/model_epoch_best_0.pth'
saved_path = Path(sp).expanduser()
state = torch.load(saved_path)
sd = {k.replace('module.model_ft.',''):v for k,v in state['state_dict'].items()}
m.load_state_dict(sd, strict=False)

_IncompatibleKeys(missing_keys=['last_linear.weight', 'last_linear.bias'], unexpected_keys=['last_linear.0.weight', 'last_linear.0.bias'])

In [ ]:
dbch = get_data(128, 256, splits=Meta.splits)
learn = get_learner(dbch, m)

## Training

In [12]:
do_fit(learn, 1, 5e-4, freeze=False)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.072386,0.073083,0.976464,0.955651,1:27:49


In [13]:
dbch = get_data(64, 384, splits=Meta.splits, img_dir=path_jpg)
do_fit(learn, 2, 1e-4, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.066882,0.073282,0.976324,0.955513,1:24:57
1,0.065809,0.071357,0.976982,0.956534,1:25:09


## Submission

In [10]:
learn.load(f'runs/{name}-2')
sub_fn = f'subm/{name}'

In [11]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=384, tst_dir='tst_jpg')

In [12]:
ob = learn.dls.one_batch()

In [13]:
preds,targs = learn.get_preds()

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [16]:
api.competition_submit(f'{sub_fn}.csv', f'{name} seutao 384 2epoch', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 11.6MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [ ]:
'Private score - 0.08553'

In [24]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13819985,
 'totalBytes': 27277209,
 'date': '2019-12-16T18:56:20.56Z',
 'description': 'baseline_any_xse_resnet_256_seutao seutao 384 2epoch',
 'errorDescription': None,
 'fileName': 'baseline_any_xse_resnet_256_seutao.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13819985/13819985.raw'}